In [17]:
#import libraries
import pandas as pd
import numpy as np
import openpyxl as op
from google.colab import files
from openpyxl.utils.dataframe import dataframe_to_rows

#create upload link
uploaded = files.upload()

Saving kaggle.json to kaggle (1).json


In [18]:
#copy json file in a directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#permission to access kaggle data
!chmod 600 ~/.kaggle/kaggle.json

#Download kaggle data
!kaggle datasets download -d wendykan/lending-club-loan-data

#Unzip kaggle data
from zipfile import ZipFile
file_name = "lending-club-loan-data.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Unzip Done')

#import kaggle Data
data = pd.read_csv('loan.csv')
total_rows_Data=len(data.axes[0])
print('Total Rows in Data:', total_rows_Data)

lending-club-loan-data.zip: Skipping, found more recently modified local copy (use --force to force download)
Unzip Done


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Total Rows in Data: 2260668


In [19]:
#Create Working Dataframe
df = data.loc[:, ['annual_inc', 'loan_amnt', 'annual_inc_joint', 'grade', 'loan_status', 'acc_now_delinq', 'dti', 'dti_joint', 'inq_last_12m', 'inq_fi', 'hardship_flag', 'collections_12_mths_ex_med']]

#Finding Total number of rows after data cleaning
total_rows = len(df.axes[0])
print('Total Rows in DataFrame',total_rows)

#Eliminating Closed Account from Data
indexNames = df[(df['loan_status'] == 'Fully Paid')].index
df.drop(indexNames , inplace=True)

#Replacing NaN value with 0
df['annual_inc_joint'] = df['annual_inc_joint'].replace(np.nan, 0)
df['dti'] = df['dti'].replace(np.nan, 0)
df['dti_joint'] = df['dti_joint'].replace(np.nan, 0)
df['inq_last_12m'] = df['inq_last_12m'].replace(np.nan, 0)
df['inq_fi'] = df['inq_fi'].replace(np.nan, 0)


#Drop All Rows with any Null/NaN/NaT Values we can't do this operation in starting because there are many nan values in inc_joint
#df = df.dropna()

#Finding Total number of rows after data cleaning
total_rows = len(df.axes[0])
print('Total Rows in DataFrame',total_rows)

#Calculate Total income, dti and inq
sumColumn = df['annual_inc'] + df['annual_inc_joint']
df['total_inc'] = sumColumn

#Rounded real values to the nearest whole dollar
#df["total_inc"] = df["total_inc"].astype(int)

df['debt'] = df['dti'] * df['annual_inc']
df['debt_joint'] = df['dti_joint'] * df['annual_inc_joint']

df['total_dti'] = (df['debt'] + df['debt_joint']) / (df['annual_inc'] + df['annual_inc_joint'])

sumColumn2 = df['inq_last_12m'] + df['inq_fi']
df['total_inq'] = sumColumn2

#Generate Bands according to total_dti
dti_band = [-1, 10.99, 15.99, 20.99, 25.99, 1100.99]
dti_bands_name =[1, 2, 3, 4, 5]
df['dti_band'] = pd.cut(df['total_dti'], dti_band, labels = dti_bands_name)

#Generate Band according to Total income
inc_bands = [-1, 45000, 60000, 80000, 110000, 10000000]
inc_bands_name =[1, 2, 3, 4, 5]
df['inc_band'] = pd.cut(df['total_inc'], inc_bands, labels = inc_bands_name)

#Generate Bands according to total_inquiry
inq_band = [-1, 0, 1, 2, 4, 100]
inq_bands_name =[1, 2, 3, 4, 5]  
df['inq_band'] = pd.cut(df['total_inq'], inq_band, labels = inq_bands_name)

#Create buraeu bad indicator
df['hardship_flag_bin'] = df['hardship_flag']
df['hardship_flag_bin'].replace(to_replace = ['Y', 'N'],
                                value = [1, 0],
                              inplace = True)

df['delinq_flag'] = df['acc_now_delinq']
df['delinq_flag'].replace(to_replace = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                                value = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                              inplace = True)

df['collection_flag'] = df['collections_12_mths_ex_med']
df['collection_flag'].replace(to_replace = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                                value = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                              inplace = True)

df['bureau_bad'] = df['hardship_flag_bin'] + df['delinq_flag'] + df['collection_flag']

df['bureau_bad'].replace(to_replace = [0, 1, 2, 3],
                                value = [0, 1, 1, 1],
                              inplace = True)
df.head()

Total Rows in DataFrame 2260668
Total Rows in DataFrame 1218716


,annual_inc,loan_amnt,annual_inc_joint,grade,loan_status,acc_now_delinq,dti,dti_joint,inq_last_12m,inq_fi,hardship_flag,collections_12_mths_ex_med,total_inc,debt,debt_joint,total_dti,total_inq,dti_band,inc_band,inq_band,hardship_flag_bin,delinq_flag,collection_flag,bureau_bad
0,55000.0,2500,0.0,C,Current,0.0,18.24,0.0,2.0,1.0,N,0.0,55000.0,1003200.0,0.0,18.24,3.0,3,2,4,0,0.0,0.0,0.0
1,90000.0,30000,0.0,D,Current,0.0,26.52,0.0,2.0,2.0,N,0.0,90000.0,2386800.0,0.0,26.52,4.0,5,4,4,0,0.0,0.0,0.0
2,59280.0,5000,0.0,D,Current,0.0,10.51,0.0,0.0,1.0,N,0.0,59280.0,623032.8,0.0,10.51,1.0,1,2,2,0,0.0,0.0,0.0
3,92000.0,4000,0.0,D,Current,0.0,16.74,0.0,3.0,2.0,N,0.0,92000.0,1540080.0,0.0,16.74,5.0,3,4,5,0,0.0,0.0,0.0
4,57250.0,30000,0.0,C,Current,0.0,26.35,0.0,0.0,1.0,N,0.0,57250.0,1508537.5,0.0,26.35,1.0,5,2,2,0,0.0,0.0,0.0


In [0]:
values, count = np.unique(df['delinq_flag'], return_counts=True)

dfTemp = pd.DataFrame()
dfTemp['Values'] = values
dfTemp['Count'] = count
dfTemp.head()
# filename = "temp.csv"
# dfTemp.to_csv(filename)
# data1 = pd.read_csv('temp.csv', low_memory=False)

,Values,Count
0,0.0,1214691
1,1.0,3996
2,NaN,1
3,NaN,1
4,NaN,1


**Loan Status of different Demographics**

In [0]:
demographic = 'dti_band'
x = df.groupby(
   ['grade', 'loan_status']
).agg( 
    {
         'loan_status': "count",
    }
)

filename = "temp.csv"
x.to_csv(filename)
df1 = pd.read_csv('temp.csv', low_memory=False)

df1 = df1.pivot(index = 'grade', columns = 'loan_status', values = 'loan_status.1')

df1 = df1.replace(np.nan, 0)

sumColumn3 = df1['Charged Off'] + df1['Does not meet the credit policy. Status:Charged Off']
df1['Charged Off1'] = sumColumn3

sumColumn5 = df1['Default'] + df1['Late (16-30 days)'] + df1['Late (31-120 days)']
df1['Deli_exc_grc_prd'] = sumColumn5

sumColumn6 = df1['Default'] + df1['Late (16-30 days)'] + df1['Late (31-120 days)'] +df1['In Grace Period']
df1['Deli_inc_grc_prd'] = sumColumn6

df1  = df1.drop(['Default', 'In Grace Period', 'Late (16-30 days)',  'Late (31-120 days)', 'Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off', 'Charged Off'], axis=1)

#Finding Total number of rows after data cleaning
total_rows = len(df.axes[0])
print('Total Rows in DataFrame',total_rows)

df1.head(10)

Total Rows in DataFrame 1218716


loan_status,Current,Charged Off1,Deli_exc_grc_prd,Deli_inc_grc_prd
grade,,,,
A,204435.0,13774.0,1583.0,2251.0
B,275455.0,51162.0,5588.0,7596.0
C,267326.0,83410.0,8976.0,12170.0
D,120666.0,59638.0,5822.0,7787.0
E,40174.0,35522.0,2565.0,3359.0
F,9006.0,14356.0,834.0,1064.0
G,2633.0,4554.0,297.0,390.0


**Bureau Bad of Different Demographics**

In [0]:
demographic = 'dti_band'

x = df.groupby(
   [demographic, 'bureau_bad']
).agg( 
    {
         'bureau_bad': "count",
    }
)

filename = "temp.csv"
x.to_csv(filename)
df1 = pd.read_csv('temp.csv', low_memory=False)

df1 = df1.pivot(index = demographic, columns = 'bureau_bad', values = 'bureau_bad.1')

df1 = df1.replace(np.nan, 0)
df1.head(10)

bureau_bad,0.0,1.0
dti_band,,
1,237162,5879
2,236080,5779
3,247741,5677
4,208950,4488
5,261616,5249
